< [Notebook 14](PartIV3.ipynb) | [PyFinLab Index](ALWAYS-START-HERE.ipynb) | [Notebook 16](PartV2.ipynb) >

<a id = "ref00"></a>

<a><img src="figures/UUBS.png" width="180" height="180" border="10" /></a>

<hr>

### Notebook 15: Multiple linear regression

In this notebook we build and test a multiple linear regression prediction model for the next-day opening price of the SPY[ETF], based on historical data from a global range of stock markets. This is the first and most critical step in the process of building a signal-based trading model for the SPY[ETF], which will be the subject of the final video and notebook.

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Step 0: Importing data into seperate data frames

In [ ]:
# All Ordinary(Australia)
aord = pd.read_csv('data/indexdata/ALLOrdinary.csv').set_index('Date')
# Nikkei(Japan)
nikkei = pd.read_csv('data/indexdata/Nikkei225.csv').set_index('Date')
# Hang Seng(Hong Kong)
hsi = pd.read_csv('data/indexdata/HSI.csv').set_index('Date')
# Dax(Germany)
daxi = pd.read_csv('data/indexdata/DAXI.csv').set_index('Date')
# CAC40(France)
cac40 = pd.read_csv('data/indexdata/CAC40.csv').set_index('Date')
# Standard and Poor's(US)
sp500 = pd.read_csv('data/indexdata/SP500.csv').set_index('Date')
# Dow Jones(US)
dji = pd.read_csv('data/indexdata/DJI.csv').set_index('Date')
# Nasdaq(US)
nasdaq = pd.read_csv('data/indexdata/NASDAQ_composite.csv').set_index('Date')
# SPY[ETF](US)
spy = pd.read_csv('data/indexdata/SPY.csv').set_index('Date')

In [ ]:
spy.head()

### Step 1: Data Munging

In [ ]:
# due to time-zone differences, we extract appropriate stock
# market data for analysis. 

# Indexpanel is the DataFrame for our trading model.
Indexpanel = pd.DataFrame(index=spy.index)

# SPY[ETF]
Indexpanel['spy']=spy['Open'].shift(-1)-spy['Open']
Indexpanel['spy_lag1']=Indexpanel['spy'].shift(1)
# US Market
Indexpanel['sp500']=sp500["Open"]-sp500['Open'].shift(1)
Indexpanel['nasdaq']=nasdaq['Open']-nasdaq['Open'].shift(1)
Indexpanel['dji']=dji['Open']-dji['Open'].shift(1)
# European Markets
Indexpanel['cac40']=cac40['Open']-cac40['Open'].shift(1)
Indexpanel['daxi']=daxi['Open']-daxi['Open'].shift(1)
# Asian/Australian Markets
Indexpanel['aord']=aord['Close']-aord['Open']
Indexpanel['hsi']=hsi['Close']-hsi['Open']
Indexpanel['nikkei']=nikkei['Close']-nikkei['Open']
# used for paper trading in next video
Indexpanel['Price']=spy['Open']

In [ ]:
Indexpanel.head()

In [ ]:
# let's check to see if we have NaN values in Indexpanel
Indexpanel.isnull().sum()

In [ ]:
# we can use the DataFrame method .fillna() to forward fill 
# the Nan values. We can then drop any remaining Nan values

Indexpanel = Indexpanel.fillna(method='ffill')
Indexpanel = Indexpanel.dropna()

In [ ]:
# let's see if we have any Nan values in Indexpanel now...
Indexpanel.isnull().sum()

In [ ]:
# save our Indexpanel data frame object to CSV for later use
Indexpanel.to_csv('data/indexdata/Indexpanel.csv')


In [ ]:
print(Indexpanel.shape)

### Step 2: Splitting the data 

In [ ]:
# split the data into non-overlapping training and test sets

Train = Indexpanel.iloc[-2000:-1000, :]
Test = Indexpanel.iloc[-1000:, :]
print(Train.shape, Test.shape)

### Step 3: Exploring the training data set

In [ ]:
# generate a scatter matrix across all stock markets 
# (and the price of SPY) to observe the association

from pandas.plotting import scatter_matrix
sm = scatter_matrix(Train,figsize=(10, 10))

### Step 4: Check the correlation of each index with spy

In [ ]:
# find the indices with the strongest association
corr_array = Train.iloc[:, :-1].corr()['spy']
print(corr_array)

### Step 5: Fit multiple linear regression model to the training data

In [ ]:
formula = 'spy~spy_lag1+sp500+nasdaq+dji+cac40+daxi+aord+hsi+nikkei'
lm = smf.ols(formula=formula, data=Train).fit()
lm.summary()

In [ ]:
# investigating multicollinearity in predictors which 'failed' the p-value test
Train.iloc[:,:-1].corr()

### Step 6: Making predictions

In [ ]:
Train['PredictedY'] = lm.predict(Train)
Test['PredictedY'] = lm.predict(Test)

In [ ]:
plt.scatter(Train['spy'], Train['PredictedY'])

### Step 7: Model evaluation - statistical standards

We can measure the performance of our model using some statistical metrics: 
<br>
**RMSE** and **Adjusted** $R^2$ 

In [ ]:
# defining our own Python function to calulate the
# RMSE(Root Mean Squared Error) and Adjusted R-squared
def adjustedMetrics(data,model,model_k,yname):
    data['yhat'] = model.predict(data)
    SST = ((data[yname]-data[yname].mean())**2).sum()
    SSR = ((data['yhat']-data[yname].mean())**2).sum()
    SSE = ((data[yname]-data['yhat'])**2).sum()
    r2 = SSR/SST
    adjRsquared = 1-(1-r2)*(data.shape[0]-1)/(data.shape[0]-model_k-1)
    RMSE = (SSE/(data.shape[0]-model_k-1))**0.5
    return adjRsquared,RMSE

In [ ]:
print('Adjusted R-squared and RMSE for Train',adjustedMetrics(Train,lm,9,'spy'))
print('Adjusted R-squared and RMSE for Test',adjustedMetrics(Test,lm,9,'spy'))

In [ ]:
def assessTable(test,train,model,model_k,yname):
    r2test,RMSEtest = adjustedMetrics(test,model,model_k,yname)
    r2train,RMSEtrain = adjustedMetrics(train,model,model_k,yname)
    assessment = pd.DataFrame(index=['R2','RMSE'],columns=['Train','Test'])
    assessment['Train'] = [r2train,RMSEtrain]
    assessment['Test'] = [r2test,RMSEtest]
    return assessment

In [ ]:
# get the assessment table for our model
assessTable(Test,Train,lm,9,'spy')

< [Notebook 14](PartIV3.ipynb) | [PyFinLab Index](ALWAYS-START-HERE.ipynb) | [Notebook 16](PartV2.ipynb) >

<div align="right"><a href="#ref00">back to top</a></div>